### Importing Packages 

In [1]:
from pathlib import Path
import os as os 
import pandas as pd
import re

In [6]:

# ---------- CONFIG ----------
BASE_PATH = Path(r"C:\\Users\\asus\Desktop\\Abdi_Asfaw_Alcon_Project\\Raw Data")
output_path = Path(r"C:\\Users\\asus\Desktop\\Abdi_Asfaw_Alcon_Project\\Processed Data")


# all Excel + CSV that look like orders and are not temp files
ORDER_FILES = [
    f for f in BASE_PATH.glob("*.*")
    if f.suffix in [".xlsx", ".csv"]
    and "order" in f.name.lower()
    and not f.name.startswith("~$")
]

print(f"Found {len(ORDER_FILES)} order files:")
for f in ORDER_FILES:
    print("  -", f.name)

Found 11 order files:
  - orders_April.xlsx
  - orders_August.xlsx
  - orders_February.xlsx
  - orders_January.xlsx
  - orders_July.xlsx
  - orders_June.xlsx
  - orders_March_bot.xlsx
  - orders_May.xlsx
  - orders_November_bot.csv
  - orders_October_bot.csv
  - orders_September.xlsx


### Custome Function to clean messy dates like (OrderDate and ActualDeliveryDate)

In [7]:
def parse_mixed_date(x):
    if pd.isna(x):
        return pd.NaT
    x = str(x).strip()
    if not x:
        return pd.NaT

    # year-first 2025-02-09 or 2025/02/09
    if re.match(r"^\d{4}[-/]", x):
        return pd.to_datetime(x, errors="coerce")

    # handle both "/" and "-"
    sep = "/" if "/" in x else "-" if "-" in x else None
    if sep:
        parts = x.split(sep)
        if len(parts) == 3 and parts[0].isdigit() and parts[1].isdigit():
            d1, d2 = map(int, parts[:2])

            if d1 > 12 and d2 <= 12:        # clearly day/month
                fmt = f"%d{sep}%m{sep}%Y"
                return pd.to_datetime(x, format=fmt, errors="coerce")
            elif d2 > 12 and d1 <= 12:      # clearly month/day
                fmt = f"%m{sep}%d{sep}%Y"
                return pd.to_datetime(x, format=fmt, errors="coerce")
            else:
                # ambiguous → default to dayfirst
                return pd.to_datetime(x, dayfirst=True, errors="coerce")

    # fallback
    return pd.to_datetime(x, errors="coerce")


In [8]:
def load_orders_file(path: Path) -> pd.DataFrame:
    """
    Reads one orders file (Excel/CSV), auto-detects headers or assigns them,
    normalizes column names, and cleans dates.
    """
    print(f"\nReading: {path.name}")

    # ---- CASE 1: CSV ----
    if path.suffix == ".csv":
        preview = pd.read_csv(path, nrows=5, header=None)
        header_row = None
        for i, row in preview.iterrows():
            if row.astype(str).str.contains("OrderID", case=False, na=False).any():
                header_row = i
                break
        df = pd.read_csv(path, header=header_row if header_row is not None else None)

    # ---- CASE 2: EXCEL ----
    else:
        preview = pd.read_excel(path, nrows=15, header=None)
        header_row = None
        for i, row in preview.iterrows():
            if row.astype(str).str.contains("OrderID", case=False, na=False).any():
                header_row = i
                break

        if header_row is not None:
            df = pd.read_excel(path, header=header_row)
        else:
            # no header found: read from data block, assign column names manually
            df = pd.read_excel(path, skiprows=9, usecols="C:J", header=None)
            df.columns = [
                "OrderID", "CustomerID", "CustomerName",
                "ProductID", "Quantity", "OrderDate",
                "RequestedDeliveryDate", "Comments"
            ]

    # ---- CLEANUP ----
    df = df.dropna(axis=1, how="all")
    df.columns = df.columns.str.strip().str.lower()

    # rename similar names
    rename_map = {
        "orderid": "OrderID",
        "customerid": "CustomerID",
        "customername": "CustomerName",
        "productid": "ProductID",
        "product": "ProductID",
        "column1": "ProductID",
        "quantity": "Quantity",
        "orderdate": "OrderDate",
        "requesteddeliverydate": "RequestedDeliveryDate",
        "comments": "Comments"
    }
    df.rename(columns=rename_map, inplace=True)

    # Ensure all expected columns exist
    expected_cols = [
        "OrderID", "CustomerID", "CustomerName",
        "ProductID", "Quantity", "OrderDate",
        "RequestedDeliveryDate", "Comments"
    ]
    for col in expected_cols:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[expected_cols]

    # ---- CLEAN TYPES ----
    for col in ["OrderDate", "RequestedDeliveryDate"]:
        df[col] = df[col].apply(parse_mixed_date)

    df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce").astype("Int64")
    df["SourceFile"] = path.name

    return df


In [9]:
# ---------- PIPELINE ----------
all_orders = []

for f in ORDER_FILES:
    try:
        df = load_orders_file(f)
        all_orders.append(df)
    except Exception as e:
        print(f"⚠️ Error reading {f.name}: {e}")

orders_all = pd.concat(all_orders, ignore_index=True)
orders_all.drop_duplicates(subset=["OrderID", "ProductID"], inplace=True)

print("\n✅ Combined orders shape:", orders_all.shape)
orders_all.head(10)



Reading: orders_April.xlsx

Reading: orders_August.xlsx

Reading: orders_February.xlsx

Reading: orders_January.xlsx

Reading: orders_July.xlsx

Reading: orders_June.xlsx

Reading: orders_March_bot.xlsx

Reading: orders_May.xlsx

Reading: orders_November_bot.csv

Reading: orders_October_bot.csv

Reading: orders_September.xlsx

✅ Combined orders shape: (88000, 9)


,OrderID,CustomerID,CustomerName,ProductID,Quantity,OrderDate,RequestedDeliveryDate,Comments,SourceFile
0,O124000,C0800,Customer 800,P0649,6,2025-11-04,2025-04-17,NaN,orders_April.xlsx
1,O124001,C0118,Customer 118,P0086,18,2025-04-20,2025-04-25,Pay in 90 days,orders_April.xlsx
2,O124002,C0254,Customer 254,P0236,14,2025-04-08,2025-04-10,NaN,orders_April.xlsx
3,O124003,C0169,Customer 169,P0492,6,2025-04-25,2025-04-28,NaN,orders_April.xlsx
4,O124004,C0388,Customer 388,P0391,17,2025-04-14,2025-04-23,NaN,orders_April.xlsx
5,O124005,C0192,Customer 192,P0292,3,2025-09-04,2025-04-13,Inform customer once order is sent,orders_April.xlsx
6,O124006,C0447,Customer 447,P0130,8,2025-04-09,2025-04-15,Inform customer once order is sent,orders_April.xlsx
7,O124007,C0890,Customer 890,P0532,18,2025-04-19,2025-04-29,Pay in 90 days,orders_April.xlsx
8,O124008,C0337,Customer 337,P0883,20,2025-04-13,2025-04-15,Pay in 90 days,orders_April.xlsx
9,O124009,C0043,Customer 43,P0006,11,2025-04-10,2025-04-12,Pay in 90 days,orders_April.xlsx


In [6]:
orders_all.dtypes

OrderID                          object
CustomerID                       object
CustomerName                     object
ProductID                        object
Quantity                          Int64
OrderDate                datetime64[ns]
RequestedDeliveryDate    datetime64[ns]
Comments                         object
SourceFile                       object
dtype: object

In [7]:
# Find CustomerIDs mapped to more than one distinct CustomerName
duplicates = (
    orders_all.groupby("CustomerID")["CustomerName"]
      .nunique()
      .reset_index()
      .query("CustomerName > 1")["CustomerID"]
)

# Filter and print all rows for those CustomerIDs
conflicting_customers = orders_all[orders_all["CustomerID"].isin(duplicates)]
print(conflicting_customers)


          OrderID CustomerID  CustomerName ProductID  Quantity  OrderDate  \
64000  ORD1100000   CUST2824   Customer_13   PROD859         9 2025-11-02   
64003  ORD1100003   CUST7924  Customer_175   PROD384         5 2025-02-11   
64004  ORD1100004   CUST5333  Customer_414   PROD144        15 2025-05-11   
64010  ORD1100010   CUST9830  Customer_135   PROD864        19 2025-11-04   
64013  ORD1100013   CUST2876  Customer_350   PROD649         9 2025-07-11   
...           ...        ...           ...       ...       ...        ...   
71993  ORD1107993   CUST4254  Customer_447   PROD673        12 2025-11-04   
71995  ORD1107995   CUST4346   Customer_62   PROD969        14 2025-02-11   
71996  ORD1107996   CUST4909  Customer_135   PROD934         7 2025-02-11   
71998  ORD1107998   CUST1042  Customer_195   PROD616         6 2025-11-05   
71999  ORD1107999   CUST8357   Customer_18   PROD245         9 2025-03-11   

      RequestedDeliveryDate                            Comments  \
64000   

In [8]:
orders_all.groupby("CustomerID")["CustomerName"].unique().loc[duplicates]


CustomerID
CUST1001    [Customer_128, Customer_219, Customer_269]
CUST1006                   [Customer_62, Customer_340]
CUST1019                  [Customer_460, Customer_468]
CUST1021                  [Customer_140, Customer_175]
CUST1023     [Customer_389, Customer_177, Customer_66]
                               ...                    
CUST9972                  [Customer_218, Customer_244]
CUST9981                   [Customer_389, Customer_97]
CUST9985     [Customer_28, Customer_337, Customer_456]
CUST9992                  [Customer_237, Customer_426]
CUST9996                   [Customer_52, Customer_163]
Name: CustomerName, Length: 2006, dtype: object

### Cleaning the Product ID 

In [27]:
def normalize_product_id(pid):
    s = str(pid)
    if s.startswith("PRODUCT-P"):
        return s[-5:]               # last 5 chars: Pxxxx
    if s.startswith("PROD"):
        return "P" + s[4:].zfill(4) # PROD859 -> P0859
    return s                        # already Pxxxx

orders_all["ProductID"] = orders_all["ProductID"].apply(normalize_product_id)

### Cleaning OrderID 

In [28]:
def normalize_order_id(order_id):
    s = str(order_id)
    if s.startswith("ORD"):
        # Keep numeric part, make it look like O + digits after removing leading 1
        num = s[3:]           # e.g. "1100000"
        if num.startswith("11"):
            num = num[1:]     # remove extra 1
        return "O" + num
    return s  # already clean

orders_all["OrderID"] = orders_all["OrderID"].apply(normalize_order_id)


### saving the cleaned and joined orders 

In [32]:
orders_all_output_path = os.path.join(output_path,"all_orders_combined.xlsx")
orders_all.to_excel(orders_all_output_path, index=False, engine="openpyxl")
print(f"✅ File saved successfully to: {orders_all_output_path}")

✅ File saved successfully to: C:\Users\asus\Desktop\Alcon\Processed Data\all_orders_combined.xlsx


In [10]:
orders_all.dtypes

OrderID                          object
CustomerID                       object
CustomerName                     object
ProductID                        object
Quantity                          Int64
OrderDate                datetime64[ns]
RequestedDeliveryDate    datetime64[ns]
Comments                         object
SourceFile                       object
dtype: object

### Product Information 

In [33]:
product_master = pd.read_excel(BASE_PATH/"product_master.xlsx")

In [34]:
product_master.rename(columns= {'ProductCode':'ProductID'}, inplace = True) # Normalizing the name columns Name to join them later 

In [35]:
product_master.isna().any()

ProductID                                   False
ProductName                                 False
FamilyHierarchy (Category > SubCategory)    False
dtype: bool

#### Saving the cleaned Product Data 

In [36]:
product_output_path = os.path.join(output_path, "product_master_cleaned.xlsx")
product_master.to_excel(product_output_path, index=False, engine="openpyxl")
print(f"✅ File saved successfully to: {product_output_path}")


✅ File saved successfully to: C:\Users\asus\Desktop\Alcon\Processed Data\product_master_cleaned.xlsx


### Deliveries Data 

In [37]:
deliveries = pd.read_excel(BASE_PATH / 'deliveries.xlsx', skiprows=8, usecols='C:H')

deliveries = deliveries[deliveries["OrderID"].astype(str).str.match(r"^O\d+", na=False)]

deliveries.reset_index(drop=True, inplace=True)

In [15]:
deliveries.tail(10)

,OrderID,ProductID,DeliveryID,Carrier,ActualDeliveryDate,DeliveryStatus
99990,O199990,P0462,D099991,UPS,2025-01-04,On Time
99991,O199991,P0148,D099992,GLS,2025-11-03,On Time
99992,O199992,P0877,D099993,UPS,27/04/2025,On Time
99993,O199993,P0245,D099994,UPS,08-16-2025,On Time
99994,O199994,P0334,D099995,UPS,28/07/2025,On Time
99995,O199995,P0320,D099996,UPS,2025-07-22,On Time
99996,O199996,P0346,D099997,GLS,2025-12-24,On Time
99997,O199997,P0118,D099998,GLS,2025-04-22,On Time
99998,O199998,P0817,D099999,UPS,09/04/2025,On Time
99999,O199999,P0403,D100000,GLS,11-08-2025,On Time


In [15]:
orders_all.head()

,OrderID,CustomerID,CustomerName,ProductID,Quantity,OrderDate,RequestedDeliveryDate,Comments,SourceFile
0,O124000,C0800,Customer 800,P0649,6,2025-04-11,2025-04-17,NaN,orders_April.xlsx
1,O124001,C0118,Customer 118,P0086,18,2025-04-20,2025-04-25,Pay in 90 days,orders_April.xlsx
2,O124002,C0254,Customer 254,P0236,14,2025-08-04,2025-10-04,NaN,orders_April.xlsx
3,O124003,C0169,Customer 169,P0492,6,2025-04-25,2025-04-28,NaN,orders_April.xlsx
4,O124004,C0388,Customer 388,P0391,17,2025-04-14,2025-04-23,NaN,orders_April.xlsx


In [14]:
deliveries.shape

(100000, 6)

In [17]:
deliveries.isna().any() 

OrderID               False
ProductID             False
DeliveryID            False
Carrier               False
ActualDeliveryDate    False
DeliveryStatus        False
dtype: bool

In [38]:
deliveries.ActualDeliveryDate = deliveries.ActualDeliveryDate.apply(parse_mixed_date)  # Cleaning the messy date to the standard and clean date 

In [17]:
deliveries.tail(10)

,OrderID,ProductID,DeliveryID,Carrier,ActualDeliveryDate,DeliveryStatus
99990,O199990,P0462,D099991,UPS,2025-01-04,On Time
99991,O199991,P0148,D099992,GLS,2025-11-03,On Time
99992,O199992,P0877,D099993,UPS,2025-04-27,On Time
99993,O199993,P0245,D099994,UPS,2025-08-16,On Time
99994,O199994,P0334,D099995,UPS,2025-07-28,On Time
99995,O199995,P0320,D099996,UPS,2025-07-22,On Time
99996,O199996,P0346,D099997,GLS,2025-12-24,On Time
99997,O199997,P0118,D099998,GLS,2025-04-22,On Time
99998,O199998,P0817,D099999,UPS,2025-09-04,On Time
99999,O199999,P0403,D100000,GLS,2025-11-08,On Time


In [20]:
deliveries.dtypes

OrderID                       object
ProductID                     object
DeliveryID                    object
Carrier                       object
ActualDeliveryDate    datetime64[ns]
DeliveryStatus                object
dtype: object

#### Saving Deliveries cleaned Data

In [39]:
deliveries_output_path = os.path.join(output_path,'deliveries_cleaned.xlsx')
deliveries.to_excel(deliveries_output_path,index = False, engine = 'openpyxl')
print(f'file saved to {deliveries_output_path}')

file saved to C:\Users\asus\Desktop\Alcon\Processed Data\deliveries_cleaned.xlsx


### Complaints 

In [40]:
complaints = pd.read_excel(BASE_PATH/'complaints.xlsx')

In [22]:
complaints.dtypes

ComplaintID    object
DeliveryID     object
OrderID        object
ProductID      object
Issue          object
dtype: object

In [23]:
complaints.isna().any()

ComplaintID    False
DeliveryID     False
OrderID        False
ProductID      False
Issue          False
dtype: bool

In [24]:
complaints.head()

,ComplaintID,DeliveryID,OrderID,ProductID,Issue
0,CPL00014,D000014,O100013,00P0136,Package damaged during transit
1,CPL00030,D000030,O100029,P0222,Package damaged during transit
2,CPL00034,D000034,O100033,P0753,Package damaged during transit
3,CPL00056,D000056,O100055,00P0848,Package damaged during transit
4,CPL00123,D000123,O100122,P0599,Package damaged during transit


In [41]:
complaints_ouput_path = os.path.join(output_path,'complaints_cleaned.xlsx')
complaints.to_excel(complaints_ouput_path,index = False, engine = 'openpyxl')
print(f'the file is saved in {complaints_ouput_path}')

the file is saved in C:\Users\asus\Desktop\Alcon\Processed Data\complaints_cleaned.xlsx


### Merging all 

In [42]:
# Merge orders with deliveries on OrderID + ProductID
orders_deliveries = pd.merge(
    orders_all, deliveries,
    on="OrderID",#["OrderID", "ProductID"],
    how="left",
    suffixes=("", "_Delivery")
)

In [20]:
orders_deliveries.head()

,OrderID,CustomerID,CustomerName,ProductID,Quantity,OrderDate,RequestedDeliveryDate,Comments,SourceFile,ProductID_Delivery,DeliveryID,Carrier,ActualDeliveryDate,DeliveryStatus
0,O124000,C0800,Customer 800,P0649,6,2025-04-11,2025-04-17,NaN,orders_April.xlsx,P0251,D024001,DHL,2025-07-05,On Time
1,O124001,C0118,Customer 118,P0086,18,2025-04-20,2025-04-25,Pay in 90 days,orders_April.xlsx,P0703,D024002,UPS,2025-08-25,On Time
2,O124002,C0254,Customer 254,P0236,14,2025-08-04,2025-10-04,NaN,orders_April.xlsx,P0690,D024003,DHL,2025-02-22,On Time
3,O124003,C0169,Customer 169,P0492,6,2025-04-25,2025-04-28,NaN,orders_April.xlsx,P0579,D024004,UPS,2025-10-05,On Time
4,O124004,C0388,Customer 388,P0391,17,2025-04-14,2025-04-23,NaN,orders_April.xlsx,P0727,D024005,DHL,2025-02-11,Delayed


In [43]:

# Merge with product master (for product details)
orders_full = pd.merge(
    orders_deliveries, product_master,
    on="ProductID",
    how="left"
)

In [44]:
orders_full.head()

,OrderID,CustomerID,CustomerName,ProductID,Quantity,OrderDate,RequestedDeliveryDate,Comments,SourceFile,ProductID_Delivery,DeliveryID,Carrier,ActualDeliveryDate,DeliveryStatus,ProductName,FamilyHierarchy (Category > SubCategory)
0,O124000,C0800,Customer 800,P0649,6,2025-11-04,2025-04-17,NaN,orders_April.xlsx,P0251,D024001,DHL,2025-05-07,On Time,Regular Product 649,Eye Wipes > Regular
1,O124001,C0118,Customer 118,P0086,18,2025-04-20,2025-04-25,Pay in 90 days,orders_April.xlsx,P0703,D024002,UPS,2025-08-25,On Time,Sensitive Product 86,Eye Wipes > Sensitive
2,O124002,C0254,Customer 254,P0236,14,2025-04-08,2025-04-10,NaN,orders_April.xlsx,P0690,D024003,DHL,2025-02-22,On Time,Steroid Product 236,Eye Allergy > Steroid
3,O124003,C0169,Customer 169,P0492,6,2025-04-25,2025-04-28,NaN,orders_April.xlsx,P0579,D024004,UPS,2025-05-10,On Time,Antihistamine Product 492,Eye Allergy > Antihistamine
4,O124004,C0388,Customer 388,P0391,17,2025-04-14,2025-04-23,NaN,orders_April.xlsx,P0727,D024005,DHL,2025-02-11,Delayed,Antibiotic Product 391,Eye Drops > Antibiotic


In [45]:
# Merge complaints (linked by OrderID)
orders_final = pd.merge(
    orders_full, complaints,
    on="OrderID",#["OrderID", "ProductID"],
    how="left",
    suffixes=("", "_Complaint")
)

In [46]:
orders_final.head()

,OrderID,CustomerID,CustomerName,ProductID,Quantity,OrderDate,RequestedDeliveryDate,Comments,SourceFile,ProductID_Delivery,DeliveryID,Carrier,ActualDeliveryDate,DeliveryStatus,ProductName,FamilyHierarchy (Category > SubCategory),ComplaintID,DeliveryID_Complaint,ProductID_Complaint,Issue
0,O124000,C0800,Customer 800,P0649,6,2025-11-04,2025-04-17,NaN,orders_April.xlsx,P0251,D024001,DHL,2025-05-07,On Time,Regular Product 649,Eye Wipes > Regular,NaN,NaN,NaN,NaN
1,O124001,C0118,Customer 118,P0086,18,2025-04-20,2025-04-25,Pay in 90 days,orders_April.xlsx,P0703,D024002,UPS,2025-08-25,On Time,Sensitive Product 86,Eye Wipes > Sensitive,NaN,NaN,NaN,NaN
2,O124002,C0254,Customer 254,P0236,14,2025-04-08,2025-04-10,NaN,orders_April.xlsx,P0690,D024003,DHL,2025-02-22,On Time,Steroid Product 236,Eye Allergy > Steroid,NaN,NaN,NaN,NaN
3,O124003,C0169,Customer 169,P0492,6,2025-04-25,2025-04-28,NaN,orders_April.xlsx,P0579,D024004,UPS,2025-05-10,On Time,Antihistamine Product 492,Eye Allergy > Antihistamine,NaN,NaN,NaN,NaN
4,O124004,C0388,Customer 388,P0391,17,2025-04-14,2025-04-23,NaN,orders_April.xlsx,P0727,D024005,DHL,2025-02-11,Delayed,Antibiotic Product 391,Eye Drops > Antibiotic,NaN,NaN,NaN,NaN


In [47]:
Final_output_path = os.path.join(output_path , "Final_Merged_Cleaned_Data.xlsx")
orders_final.to_excel(Final_output_path, index=False, engine="openpyxl")

print(f"✅ File saved successfully to: {Final_output_path}")

✅ File saved successfully to: C:\Users\asus\Desktop\Alcon\Processed Data\Final_Merged_Cleaned_Data.xlsx


In [33]:
print(os.getcwd())

c:\Users\asus\Desktop\Alcon


In [49]:
# 1) Get max OrderID from orders (numeric part)
orders_all["order_num"] = orders_all["OrderID"].str.replace("O", "", regex=False).astype(int)
max_order_num = orders_all["order_num"].max()

print("Max order id in orders:", max_order_num)   # should be 171999

# 2) Extract deliveries that are beyond this range (O171999)
deliveries["order_num"] = deliveries["OrderID"].str.replace("O", "", regex=False).astype(int)
deliveries_out_of_range = deliveries[deliveries["order_num"] > max_order_num]

# 3) (Optional) deliveries that don’t match any order at all (safety check)
#deliveries_unmatched = deliveries[~deliveries["OrderID"].isin(orders_all["OrderID"])]

path_deliveries_missing_orders = os.path.join(output_path,"deliveries_missing_orders.xlsx")

# 4) Save for documentation / Power BI
deliveries_out_of_range.to_excel(path_deliveries_missing_orders, index=False)
#deliveries_unmatched.to_excel("deliveries_unmatched_all.xlsx", index=False)

Max order id in orders: 179999


In [50]:
orders_for_merge = (
    orders_all[["OrderID"]]
    .drop_duplicates()                 
    .rename(columns={"OrderID": "OrderID_from_Orders"})
)

merged = deliveries.merge(
    orders_for_merge,
    left_on="OrderID",
    right_on="OrderID_from_Orders",
    how="left"
)

print(len(merged))   # should now be 100000

# Check: NaN here = no matching order
print(merged["OrderID_from_Orders"].isna().sum(), "deliveries have no matching order.")
print(merged.head())

100000
20000 deliveries have no matching order.
   OrderID ProductID DeliveryID Carrier ActualDeliveryDate DeliveryStatus  \
0  O100000     P0574    D000001   FedEx         2025-09-26        On Time   
1  O100001     P0059    D000002     GLS         2025-06-28        Delayed   
2  O100002     P0045    D000003     GLS         2025-01-16        On Time   
3  O100003     P0768    D000004     DHL         2025-01-10        On Time   
4  O100004     P0671    D000005     DHL         2025-06-07        Delayed   

   order_num OrderID_from_Orders  
0     100000             O100000  
1     100001             O100001  
2     100002             O100002  
3     100003             O100003  
4     100004             O100004  


In [51]:
merged.head()

,OrderID,ProductID,DeliveryID,Carrier,ActualDeliveryDate,DeliveryStatus,order_num,OrderID_from_Orders
0,O100000,P0574,D000001,FedEx,2025-09-26,On Time,100000,O100000
1,O100001,P0059,D000002,GLS,2025-06-28,Delayed,100001,O100001
2,O100002,P0045,D000003,GLS,2025-01-16,On Time,100002,O100002
3,O100003,P0768,D000004,DHL,2025-01-10,On Time,100003,O100003
4,O100004,P0671,D000005,DHL,2025-06-07,Delayed,100004,O100004


In [52]:
product_master.head()

,ProductID,ProductName,FamilyHierarchy (Category > SubCategory)
0,P0001,Lubricating Product 1,Eye Drops > Lubricating
1,P0002,Lubricating Product 2,Eye Drops > Lubricating
2,P0003,Steroid Product 3,Eye Allergy > Steroid
3,P0004,Lubricating Product 4,Eye Drops > Lubricating
4,P0005,Antihistamine Product 5,Eye Allergy > Antihistamine


In [54]:

# Merge with product master (for product details)
all_deliveries_with_orderinfo = pd.merge(
    merged, product_master,
    on="ProductID",
    how="left"
)

In [55]:
all_deliveries_with_orderinfo.head()

,OrderID,ProductID,DeliveryID,Carrier,ActualDeliveryDate,DeliveryStatus,order_num,OrderID_from_Orders,ProductName,FamilyHierarchy (Category > SubCategory)
0,O100000,P0574,D000001,FedEx,2025-09-26,On Time,100000,O100000,Regular Product 574,Eye Wipes > Regular
1,O100001,P0059,D000002,GLS,2025-06-28,Delayed,100001,O100001,Advanced Product 59,Eye Vitamins > Advanced
2,O100002,P0045,D000003,GLS,2025-01-16,On Time,100002,O100002,Regular Product 45,Eye Wipes > Regular
3,O100003,P0768,D000004,DHL,2025-01-10,On Time,100003,O100003,Steroid Product 768,Eye Allergy > Steroid
4,O100004,P0671,D000005,DHL,2025-06-07,Delayed,100004,O100004,Steroid Product 671,Eye Allergy > Steroid


In [53]:
complaints.head()

,ComplaintID,DeliveryID,OrderID,ProductID,Issue
0,CPL00014,D000014,O100013,00P0136,Package damaged during transit
1,CPL00030,D000030,O100029,P0222,Package damaged during transit
2,CPL00034,D000034,O100033,P0753,Package damaged during transit
3,CPL00056,D000056,O100055,00P0848,Package damaged during transit
4,CPL00123,D000123,O100122,P0599,Package damaged during transit


In [56]:
all_deliveries_with_orderinfo = all_deliveries_with_orderinfo.merge(
    complaints[["OrderID", "Issue"]],
    on="OrderID",
    how="left"
).rename(columns={"Issue": "Complaints"})


In [59]:
all_deliveries_with_orderinfo.head()

,OrderID,ProductID,DeliveryID,Carrier,ActualDeliveryDate,DeliveryStatus,order_num,OrderID_from_Orders,ProductName,FamilyHierarchy (Category > SubCategory),Complaints
0,O100000,P0574,D000001,FedEx,2025-09-26,On Time,100000,O100000,Regular Product 574,Eye Wipes > Regular,NaN
1,O100001,P0059,D000002,GLS,2025-06-28,Delayed,100001,O100001,Advanced Product 59,Eye Vitamins > Advanced,NaN
2,O100002,P0045,D000003,GLS,2025-01-16,On Time,100002,O100002,Regular Product 45,Eye Wipes > Regular,NaN
3,O100003,P0768,D000004,DHL,2025-01-10,On Time,100003,O100003,Steroid Product 768,Eye Allergy > Steroid,NaN
4,O100004,P0671,D000005,DHL,2025-06-07,Delayed,100004,O100004,Steroid Product 671,Eye Allergy > Steroid,NaN


In [61]:

path_all = os.path.join(output_path,"All_deliveries_with_OrderID_from_Orders.xlsx")
# Save
all_deliveries_with_orderinfo.to_excel(path_all, index=False)
